In [7]:
import pandas
import pickle
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import matplotlib.pyplot as plt
import numpy
from random import sample
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import imagenet_utils
from keras.applications.vgg19 import VGG19, decode_predictions
from keras import Model

In [8]:
#truth
truthDf = pandas.read_json(r"data\truth.jsonl", lines=True)
truthDf = truthDf.sort_values(by=["id"], ascending = True).reset_index().drop(columns=["index"])
truthDf.head()

,truthJudgments,truthMean,id,truthClass,truthMedian,truthMode
0,"[0.0, 0.0, 0.0, 0.33333333330000003, 0.0]",0.066667,804113781580328960,no-clickbait,0.000000,0.0
1,"[1.0, 0.0, 0.6666666666000001, 1.0, 1.0]",0.733333,804119512010424320,clickbait,1.000000,1.0
2,"[0.6666666666000001, 0.6666666666000001, 0.0, ...",0.333333,804121272967983104,no-clickbait,0.333333,0.0
3,"[0.0, 0.0, 0.0, 0.33333333330000003, 0.0]",0.066667,804123103995580416,no-clickbait,0.000000,0.0
4,"[0.0, 0.6666666666000001, 0.0, 0.0, 0.0]",0.133333,804126501117435904,no-clickbait,0.000000,0.0


In [9]:
#instances
instancesDf = pandas.read_json(r"data\instances.jsonl", lines=True)
instancesDf = instancesDf.sort_values(by=["id"], ascending = True).reset_index().drop(columns=["index"])
instancesDf.head()

,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription
0,[],"[Panama Papers: Europol links 3,500 names to s...",804113781580328960,[Europol],[Law enforcement agency analysis uncovers prob...,"Panama Papers: Europol links 3,500 names to su...",Thu Dec 01 00:03:59 +0000 2016,"European Union,Panama Papers,Organised crime,W...",Law enforcement agency analysis uncovers proba...
1,[media/photo_804119509338640385.jpg],[The key to truly great chicken soup],804119512010424320,"[Chicken Noodle Soup Warm and comforting, this...","[Warm and comforting, this soup is a classic t...",A Superior Chicken Soup,Thu Dec 01 00:26:46 +0000 2016,"Cooking and Cookbooks,Soups,Poultry",For the best rendition of this American classi...
2,[],[Afghan policewomen face down their fears to s...,804121272967983104,[The battle to recruit women to the Afghan Pol...,[Afghanistan is one of the most dangerous plac...,100 Women 2016: On the frontline with the wome...,Thu Dec 01 00:33:45 +0000 2016,,The Afghan women risking all to join the police.
3,[],[Conservatives are watching less football this...,804123103995580416,"[BuzzFeed Hires Mondelez Marketing Executive, ...",[Explanations for this year’s sagging National...,Older Viewers and Conservatives Are Watching L...,Thu Dec 01 00:41:02 +0000 2016,"nfl,ratings,american football,political,genera...","Many factors are dragging down NFL ratings, in..."
4,[media/photo_804125377400553474.jpg],[Richard Sherman weighs in on Cam Newton’s str...,804126501117435904,[Richard Sherman Says 'Karma' Caught Up to Cam...,[Seattle Seahawks cornerback Richard Sherman m...,Seattle Seahawks Richard Sherman Says 'Karma' ...,Thu Dec 01 00:54:32 +0000 2016,"Football, NFL, NFC West, Seattle Seahawks, Bre...","Seattle Seahawks news, rumors, scores, schedul..."


In [10]:
#tokenize words. Remove punctuation. We didn't remove stop words because the models we'll use will take care of them for us
def createWordsTokens(row):
    if isinstance(row, list):
        row = " ".join(row)
    punctuation = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~'
    row = ''.join([c for c in row if c not in punctuation])
    wordTokens = row.split()
    return wordTokens

instancesDf["ptWordTokens"] = instancesDf["postText"].apply(lambda row : createWordsTokens(row))
instancesDf["tdWordTokens"] = instancesDf["targetDescription"].apply(lambda row : createWordsTokens(row))
instancesDf["ttWordTokens"] = instancesDf["targetTitle"].apply(lambda row : createWordsTokens(row))
instancesDf["tcWordTokens"] = instancesDf["targetCaptions"].apply(lambda row : createWordsTokens(row))
instancesDf["tpWordTokens"] = instancesDf["targetParagraphs"].apply(lambda row : createWordsTokens(row))
instancesDf.head()

,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription,ptWordTokens,tdWordTokens,ttWordTokens,tcWordTokens,tpWordTokens
0,[],"[Panama Papers: Europol links 3,500 names to s...",804113781580328960,[Europol],[Law enforcement agency analysis uncovers prob...,"Panama Papers: Europol links 3,500 names to su...",Thu Dec 01 00:03:59 +0000 2016,"European Union,Panama Papers,Organised crime,W...",Law enforcement agency analysis uncovers proba...,"[Panama, Papers, Europol, links, 3500, names, ...","[Law, enforcement, agency, analysis, uncovers,...","[Panama, Papers, Europol, links, 3500, names, ...",[Europol],"[Law, enforcement, agency, analysis, uncovers,..."
1,[media/photo_804119509338640385.jpg],[The key to truly great chicken soup],804119512010424320,"[Chicken Noodle Soup Warm and comforting, this...","[Warm and comforting, this soup is a classic t...",A Superior Chicken Soup,Thu Dec 01 00:26:46 +0000 2016,"Cooking and Cookbooks,Soups,Poultry",For the best rendition of this American classi...,"[The, key, to, truly, great, chicken, soup]","[For, the, best, rendition, of, this, American...","[A, Superior, Chicken, Soup]","[Chicken, Noodle, Soup, Warm, and, comforting,...","[Warm, and, comforting, this, soup, is, a, cla..."
2,[],[Afghan policewomen face down their fears to s...,804121272967983104,[The battle to recruit women to the Afghan Pol...,[Afghanistan is one of the most dangerous plac...,100 Women 2016: On the frontline with the wome...,Thu Dec 01 00:33:45 +0000 2016,,The Afghan women risking all to join the police.,"[Afghan, policewomen, face, down, their, fears...","[The, Afghan, women, risking, all, to, join, t...","[100, Women, 2016, On, the, frontline, with, t...","[The, battle, to, recruit, women, to, the, Afg...","[Afghanistan, is, one, of, the, most, dangerou..."
3,[],[Conservatives are watching less football this...,804123103995580416,"[BuzzFeed Hires Mondelez Marketing Executive, ...",[Explanations for this year’s sagging National...,Older Viewers and Conservatives Are Watching L...,Thu Dec 01 00:41:02 +0000 2016,"nfl,ratings,american football,political,genera...","Many factors are dragging down NFL ratings, in...","[Conservatives, are, watching, less, football,...","[Many, factors, are, dragging, down, NFL, rati...","[Older, Viewers, and, Conservatives, Are, Watc...","[BuzzFeed, Hires, Mondelez, Marketing, Executi...","[Explanations, for, this, year’s, sagging, Nat..."
4,[media/photo_804125377400553474.jpg],[Richard Sherman weighs in on Cam Newton’s str...,804126501117435904,[Richard Sherman Says 'Karma' Caught Up to Cam...,[Seattle Seahawks cornerback Richard Sherman m...,Seattle Seahawks Richard Sherman Says 'Karma' ...,Thu Dec 01 00:54:32 +0000 2016,"Football, NFL, NFC West, Seattle Seahawks, Bre...","Seattle Seahawks news, rumors, scores, schedul...","[Richard, Sherman, weighs, in, on, Cam, Newton...","[Seattle, Seahawks, news, rumors, scores, sche...","[Seattle, Seahawks, Richard, Sherman, Says, 'K...","[Richard, Sherman, Says, 'Karma', Caught, Up, ...","[Seattle, Seahawks, cornerback, Richard, Sherm..."


# Split
This will be used later. We need the train ids and the test ids

In [11]:
#get list of ids
totalIds = instancesDf["id"].to_list()
totalIds[0:9]

[804113781580328960,
 804119512010424320,
 804121272967983104,
 804123103995580416,
 804126501117435904,
 804126768105930752,
 804129660661596160,
 804130694117003264,
 804131001710546944]

In [17]:
#create train ids and test ids. We need thsi cause we have multiple inputs and the ids have to match
totalLen = len(totalIds)
trainIndexes = sample(totalIds, int(totalLen*0.7))
testIndexes = [item for item in totalIds if item not in trainIndexes]
print("train: " + str(len(trainIndexes)) + "(" + str(round(len(trainIndexes)/totalLen*100,0)) + "%)" +
"     test: " + str(len(testIndexes)) + "(" + str(round(len(testIndexes)/totalLen*100, 0)) + "%)")

train: 13676(70.0%)     test: 5862(30.0%)


In [89]:
pickle.dump(trainIndexes, open("binaries/trainIndexes.pkl", 'wb'))
pickle.dump(testIndexes, open("binaries/testIndexes.pkl", 'wb'))

# WORD2VEC

In [36]:
#get embeddings of words from pre-trained model
word_vectors = KeyedVectors.load_word2vec_format(r"binaries\GoogleNews-vectors-negative300.bin", binary=True)
word_vectors["Hello"]

array([-0.05102539,  0.12060547, -0.01257324,  0.03088379, -0.08544922,
        0.0534668 , -0.13964844, -0.16308594,  0.08300781,  0.20507812,
       -0.09423828,  0.15527344, -0.00418091,  0.02111816, -0.10986328,
        0.22460938,  0.22265625,  0.15917969,  0.05786133, -0.15527344,
        0.23046875,  0.34765625,  0.44335938, -0.14550781,  0.15136719,
        0.02209473, -0.16308594,  0.2578125 ,  0.20019531,  0.06054688,
       -0.0222168 , -0.13183594, -0.06396484, -0.12792969,  0.09814453,
       -0.13574219,  0.01928711,  0.25195312,  0.14257812,  0.2421875 ,
        0.03173828, -0.17089844,  0.26171875,  0.4296875 ,  0.24804688,
       -0.10351562, -0.13867188, -0.14257812, -0.20703125, -0.00860596,
       -0.43359375, -0.01757812,  0.40234375,  0.32617188,  0.33203125,
        0.01000977, -0.11523438, -0.12695312,  0.01940918, -0.25390625,
        0.00393677, -0.00344849, -0.19042969,  0.05419922,  0.14453125,
       -0.0546875 , -0.11132812,  0.15136719, -0.265625  ,  0.20

In [41]:
#generate embeddings for each words of each post. Maximum 20 words
def getWordVecs(row):
    newList=[]
    for word in row:
        try:
            newList.append(word_vectors[word])
        except:
            None # Word not in embedding dictionary
        if len(newList)>=20: #we only need 20 words
            return newList
    
    #if it didn't return till here, it doesnt have 20 elements yet. Fill with zeros
    for i in range(len(newList), 20):
        newList.append(numpy.zeros(300))
    return newList

instancesDf["wEmbedding"] = instancesDf["ptWordTokens"].apply(lambda row : getWordVecs(row))
instancesDf[["postText", "ptWordTokens", "wEmbedding"]].head()

,postText,ptWordTokens,wEmbedding
0,"[Panama Papers: Europol links 3,500 names to s...","[Panama, Papers, Europol, links, 3500, names, ...","[[-0.021118164, -0.06542969, -0.18164062, 0.03..."
1,[The key to truly great chicken soup],"[The, key, to, truly, great, chicken, soup]","[[-0.17285156, 0.27929688, 0.106933594, -0.158..."
2,[Afghan policewomen face down their fears to s...,"[Afghan, policewomen, face, down, their, fears...","[[-0.2421875, -0.016479492, 0.18847656, 0.1962..."
3,[Conservatives are watching less football this...,"[Conservatives, are, watching, less, football,...","[[0.029052734, -0.12597656, 0.15429688, 0.375,..."
4,[Richard Sherman weighs in on Cam Newton’s str...,"[Richard, Sherman, weighs, in, on, Cam, Newton...","[[0.14648438, 0.0014343262, -0.14453125, -0.00..."


In [42]:
#create wordInputs list
wordsInput = instancesDf["wEmbedding"].tolist()
print("Shape: " + str(len(wordsInput[0]))+","+str(wordsInput[0][0].shape))
wordsInput[0:4]

Shape: 20,(300,)


5469e-02,
         -1.81640625e-01,  4.68750000e-02, -2.96875000e-01, -1.44531250e-01,
          2.89062500e-01, -3.14941406e-02,  1.12304688e-01, -2.61230469e-02,
          1.38854980e-03,  1.56250000e-01, -2.69531250e-01,  1.26953125e-01,
          1.63085938e-01,  7.17773438e-02, -2.67578125e-01,  1.06933594e-01,
         -2.69775391e-02, -9.61914062e-02, -1.61132812e-02,  2.16796875e-01,
          1.00097656e-01, -3.20312500e-01, -7.91015625e-02, -6.89697266e-03,
         -7.37304688e-02, -3.49121094e-02,  6.39648438e-02, -1.32446289e-02,
         -9.32617188e-02,  4.66308594e-02, -3.57421875e-01, -5.15747070e-03,
          1.10351562e-01, -1.29882812e-01, -1.63085938e-01,  2.69775391e-02,
         -1.10839844e-01,  2.37304688e-01,  2.05078125e-01,  9.91210938e-02,
          1.20605469e-01, -2.37304688e-01,  5.95092773e-03, -8.30078125e-02,
         -1.35742188e-01, -1.98242188e-01, -4.93164062e-02,  2.39257812e-01,
         -1.56250000e-01,  1.07910156e-01, -1.62109375e-01,  2.421

In [43]:
#get train and test from wordinputs (with the right indexes)
wordsInputTrain = instancesDf[instancesDf['id'].isin(trainIndexes)]["wEmbedding"].tolist()
wordsInputTest = instancesDf[instancesDf['id'].isin(testIndexes)]["wEmbedding"].tolist()

In [40]:
#save wordinputs to pickle
pickle.dump(wordsInputTrain, open("binaries/wordsInputTrain.pkl", 'wb'))
pickle.dump(wordsInputTest, open("binaries/wordsInputTest.pkl", 'wb'))

# DOC2VEC

In [102]:
#generate tagged documents. We'll use 4 documents for each post, in order to better train the model.

taggedDocuments= []

for index, row in instancesDf.iterrows():
    taggedDocuments.append(TaggedDocument(words=row["ptWordTokens"], tags=['pt_' + str(row["id"])]))
    taggedDocuments.append(TaggedDocument(words=row["tdWordTokens"], tags=['td_' + str(row["id"])]))
    #use more relevant text to improve the model
    taggedDocuments.append(TaggedDocument(words=row["ttWordTokens"], tags=['tt_' + str(row["id"])]))
    taggedDocuments.append(TaggedDocument(words=row["tpWordTokens"], tags=['tp_' + str(row["id"])]))

taggedDocuments[0:9]

[TaggedDocument(words=['Panama', 'Papers', 'Europol', 'links', '3500', 'names', 'to', 'suspected', 'criminals'], tags=['pt_804113781580328960']),
 TaggedDocument(words=['Law', 'enforcement', 'agency', 'analysis', 'uncovers', 'probable', 'matches', 'connected', 'to', 'terrorism', 'money', 'laundering', 'and', 'organised', 'crime'], tags=['td_804113781580328960']),
 TaggedDocument(words=['Panama', 'Papers', 'Europol', 'links', '3500', 'names', 'to', 'suspected', 'criminals'], tags=['tt_804113781580328960']),
 TaggedDocument(words=['Law', 'enforcement', 'agency', 'analysis', 'uncovers', 'probable', 'matches', 'connected', 'to', 'terrorism', 'money', 'laundering', 'and', 'organised', 'crime', 'Almost', '3500', 'individuals', 'and', 'companies', 'in', 'the', 'Panama', 'Papers', 'are', 'probable', 'matches', 'for', 'suspected', 'criminals', 'including', 'terrorists', 'cybercriminals', 'and', 'cigarette', 'smugglers', 'according', 'to', 'a', 'document', 'seen', 'by', 'the', 'Guardian', 'The',

In [113]:
# Train doc2vec model - Run only once. This takes a long time!
vector_size = 300
max_epochs = 10

model = Doc2Vec(vector_size=vector_size, alpha=0.025, min_alpha=0.025, min_count=0)
model.build_vocab(taggedDocuments)
for epoch in range(max_epochs):
    print('epoch ' + str(epoch))
    model.train(taggedDocuments,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [114]:
print(model.docvecs["pt_"+str(totalIds[0])])

model.save("binaries/docModel.d2v")

[ 0.17153263  0.04772166 -0.03229137 -0.38463593 -0.07898074  0.33678788
 -0.19668435  0.24740963 -0.18152006  0.28635544  0.11556382 -0.14178896
 -0.17527841 -0.1371888  -0.09069099  0.3382316   0.49927413 -0.02677713
  0.12057696 -0.08046931  0.19330077 -0.06223743 -0.1835499  -0.0326109
  0.13576275 -0.20044236  0.18533514 -0.23550253 -0.01691064 -0.19712907
 -0.28100276  0.21513352 -0.33323956 -0.04164119  0.04689311 -0.10317971
  0.08195734  0.4510489   0.37375095 -0.38068745 -0.6383691  -0.18341313
  0.41853735  0.3951616   0.08173611  0.0405745   0.12530331 -0.05089106
  0.4073627  -0.18494931  0.05363579 -0.47667322  0.04930605  0.08184695
 -0.14973922  0.01086683  0.35989538 -0.29198822  0.22074513  0.0110029
  0.13387807  0.03990279 -0.01968911  0.1851013  -0.05029407  0.10479826
  0.32215053  0.06236199 -0.5448705   0.19848631 -0.08144476 -0.15475754
 -0.33050084 -0.35921264  0.18554178 -0.407445   -0.09105335 -0.4563737
  0.17262255 -0.15485175  0.03405532  0.02455505  0.26

In [12]:
#load model if it exists
model = Doc2Vec.load("binaries/docModel.d2v")

#get document vector from the model, for post text and targetDescription
instancesDf["ptEmbedding"] = instancesDf[["id"]].apply(lambda row : model.docvecs["pt_"+str(row[0])], axis=1) 
instancesDf["tdEmbedding"] = instancesDf[["id"]].apply(lambda row : model.docvecs["td_"+str(row[0])], axis=1)

instancesDf[["id", "postText", "ptEmbedding", "targetDescription", "tdEmbedding"]]

,id,postText,ptEmbedding,targetDescription,tdEmbedding
0,804113781580328960,"[Panama Papers: Europol links 3,500 names to s...","[0.17153263, 0.047721658, -0.032291368, -0.384...",Law enforcement agency analysis uncovers proba...,"[0.18630086, -0.16751397, -0.15527701, -0.3248..."
1,804119512010424320,[The key to truly great chicken soup],"[0.34307212, 0.2663471, -0.21023557, -0.107252...",For the best rendition of this American classi...,"[0.40207833, 0.09169146, -0.13987862, -0.23077..."
2,804121272967983104,[Afghan policewomen face down their fears to s...,"[0.0018796567, 0.054279476, 0.31657192, 0.1075...",The Afghan women risking all to join the police.,"[0.2636815, -0.20044923, 0.05749943, 0.0273467..."
3,804123103995580416,[Conservatives are watching less football this...,"[-0.15385677, -0.22928531, 0.19694602, -0.3893...","Many factors are dragging down NFL ratings, in...","[-0.060980454, -0.40748948, -0.7688499, -0.662..."
4,804126501117435904,[Richard Sherman weighs in on Cam Newton’s str...,"[0.15808621, 0.028167, 0.12358521, -0.09619520...","Seattle Seahawks news, rumors, scores, schedul...","[0.08605767, -0.06142029, -0.31274286, -0.6818..."
...,...,...,...,...,...
19533,858455355948384256,[Trump now agrees with the majority of America...,"[0.32056072, 0.062383603, -0.51736563, -0.3755...",Donald Trump spent a great portion of 2016 ins...,"[-0.2091352, -0.6762934, -0.7266204, -1.202224..."
19534,858459539296980992,[Trump has flip-flopped. But his supporters ar...,"[0.77466464, 0.07820402, -0.65979546, -0.25709...",Barely over a tenth of Trump voters think his ...,"[0.24495552, -0.2633511, -0.13568722, -0.33954..."
19535,858460992073863168,[Inside North Korea's secret prisons],"[0.13309573, -0.052835807, -0.1235173, -0.2101...",A female guard (stock photo) at a North Korean...,"[1.0498183, -0.099413924, -0.83251196, 0.21273..."
19536,858462320779026432,[UK’s response to modern slavery leaving victi...,"[0.07296889, 0.1664854, -0.028877584, -0.36811...",“Inexcusable” failures in the UK’s system for ...,"[0.5645722, 0.39497322, -0.2976324, -0.3212662..."


In [20]:
#Create train and test inputs, using train and test ids
documentsPostInputTrain = instancesDf[instancesDf['id'].isin(trainIndexes)]["ptEmbedding"].tolist()
documentsTargetInputTrain = instancesDf[instancesDf['id'].isin(trainIndexes)]["tdEmbedding"].tolist()
documentsPostInputTest = instancesDf[instancesDf['id'].isin(testIndexes)]["ptEmbedding"].tolist()
documentsTargetInputTest = instancesDf[instancesDf['id'].isin(testIndexes)]["tdEmbedding"].tolist()

In [21]:
#output to pickle
pickle.dump(documentsPostInputTrain, open("binaries/documentsPostInputTrain.pkl", 'wb'))
pickle.dump(documentsTargetInputTrain, open("binaries/documentsTargetInputTrain.pkl", 'wb'))

pickle.dump(documentsPostInputTest, open("binaries/documentsPostInputTest.pkl", 'wb'))
pickle.dump(documentsTargetInputTest, open("binaries/documentsTargetInputTest.pkl", 'wb'))

# IMAGES

In [46]:
#Load images to matrices

def imgMatrice(row):
    if not row:
        return None #return none if no image
    filename = row[0]
    img = load_img("data/"+filename,  target_size = (224, 224)) 
    img = img_to_array(img)
    #expand 1 dimension, required by vgg model
    img = img.reshape((1,) + img.shape)
    #preprocessing will scale pixels between 0 and 1 and then will normalize each channel with respect to the ImageNet dataset.
    img = imagenet_utils.preprocess_input(img) 
    return img

instancesDf["imgMatrice"]=instancesDf["postMedia"].apply(lambda row : imgMatrice(row))
instancesDf[["postMedia", "imgMatrice"]].head()

,postMedia,imgMatrice
0,[],None
1,[media/photo_804119509338640385.jpg],"[[[[141.061 130.22101 124.32 ], [141.061 ..."
2,[],None
3,[],None
4,[media/photo_804125377400553474.jpg],"[[[[ -84.939 -97.779 -104.68 ], [ -84.939 -9..."


In [47]:
#example
instancesDf["imgMatrice"][1].shape

(1, 224, 224, 3)

In [48]:
# Import VGG19 Model and create a new model
vgg = VGG19(include_top=True) #include the top dense layers
model = Model(vgg.input, vgg.output)

In [49]:
#example
model.predict(instancesDf["imgMatrice"][1])[0]

array([7.37001127e-09, 1.08894506e-06, 8.52247883e-10, 1.26938487e-10,
       1.80927703e-10, 1.53711799e-08, 2.54399768e-09, 1.54720767e-11,
       1.41099202e-10, 8.24914997e-11, 2.15442664e-09, 2.89638286e-10,
       1.75341830e-09, 1.74813219e-09, 8.16518742e-10, 1.25421384e-09,
       7.13699211e-10, 2.69855194e-10, 9.95284133e-11, 7.02682579e-10,
       8.25782775e-10, 1.25086150e-10, 5.92673003e-12, 2.24026839e-11,
       6.97519820e-11, 2.19575846e-09, 9.52303907e-08, 6.11848208e-08,
       9.08191833e-09, 1.29039703e-08, 7.75786713e-10, 2.36310456e-08,
       6.78376066e-09, 8.78099427e-10, 1.93719846e-10, 1.32342337e-09,
       1.47126995e-08, 8.01786415e-10, 3.96771727e-09, 3.11154191e-09,
       2.63006630e-08, 3.65238195e-09, 7.32024552e-10, 1.07241571e-09,
       8.09115941e-10, 2.53169652e-10, 1.09717382e-08, 1.19235066e-08,
       1.64997238e-10, 6.57185861e-10, 3.20099308e-10, 3.27858407e-10,
       1.96716554e-09, 1.62400238e-09, 6.05417605e-10, 6.30506813e-10,
      

In [50]:
#Predict using model to get embeddings for each image
def getImageEmbeddings(row):
    if row is None:
        return numpy.zeros(1000)
    return model.predict(row)[0]

instancesDf["imgEmbeddings"]=instancesDf["imgMatrice"].apply(lambda row : getImageEmbeddings(row))
instancesDf[["postMedia", "imgMatrice", "imgEmbeddings"]].head()

,postMedia,imgMatrice,imgEmbeddings
0,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,[media/photo_804119509338640385.jpg],"[[[[141.061 130.22101 124.32 ], [141.061 ...","[7.3700113e-09, 1.088945e-06, 8.522479e-10, 1...."
2,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,[media/photo_804125377400553474.jpg],"[[[[ -84.939 -97.779 -104.68 ], [ -84.939 -9...","[3.7560321e-06, 1.1203034e-06, 3.1840239e-06, ..."


In [51]:
#get features to feature description. Get only most probable feature
instancesDf["imgFeatures"]=instancesDf["imgEmbeddings"].apply(lambda row : decode_predictions(numpy.array([row]))[0][0][1])
instancesDf[["postMedia", "imgMatrice", "imgEmbeddings", "imgFeatures"]]

,postMedia,imgMatrice,imgEmbeddings,imgFeatures
0,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",toilet_tissue
1,[media/photo_804119509338640385.jpg],"[[[[141.061 130.22101 124.32 ], [141.061 ...","[7.3700113e-09, 1.088945e-06, 8.522479e-10, 1....",consomme
2,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",toilet_tissue
3,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",toilet_tissue
4,[media/photo_804125377400553474.jpg],"[[[[ -84.939 -97.779 -104.68 ], [ -84.939 -9...","[3.7560321e-06, 1.1203034e-06, 3.1840239e-06, ...",rugby_ball
...,...,...,...,...
19533,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",toilet_tissue
19534,[media/photo_858459536574828544.jpg],"[[[[-103.939 -116.779 -123.68 ], [-103.939 -11...","[2.830179e-06, 3.861402e-07, 1.6875177e-06, 2....",shield
19535,"[media/photo_858460986612862976.jpg, media/pho...","[[[[55.060997 22.221 -1.6800003], [55.060...","[3.848476e-05, 9.711091e-07, 3.2865377e-05, 2....",military_uniform
19536,[],None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",toilet_tissue


In [52]:
# Get train and test data according to train and test indexes
imagesInputTrain = instancesDf[instancesDf['id'].isin(trainIndexes)]["imgEmbeddings"].tolist()
imagesInputTest = instancesDf[instancesDf['id'].isin(testIndexes)]["imgEmbeddings"].tolist()

In [155]:
pickle.dump(imagesInputTrain, open("binaries/imagesInputTrain.pkl", 'wb'))
pickle.dump(imagesInputTest, open("binaries/imagesInputTest.pkl", 'wb'))

# OUTPUT

In [53]:
#get the truth column, clickbait or not, to a list. Train and test, according to train and text indexes
truthDic = {"clickbait":1, "no-clickbait":0}
truthDf["output"] = truthDf["truthClass"].apply(lambda row : truthDic[row])
truthDf[["id", "truthMean", "truthClass", "output"]].head()

,id,truthMean,truthClass,output
0,804113781580328960,0.066667,no-clickbait,0
1,804119512010424320,0.733333,clickbait,1
2,804121272967983104,0.333333,no-clickbait,0
3,804123103995580416,0.066667,no-clickbait,0
4,804126501117435904,0.133333,no-clickbait,0


In [54]:
outputTrain = truthDf[truthDf['id'].isin(trainIndexes)]["output"].to_list()
outputTest = truthDf[truthDf['id'].isin(testIndexes)]["output"].to_list()

In [16]:
pickle.dump(outputTrain, open("binaries/outputTrain.pkl", 'wb'))
pickle.dump(outputTest, open("binaries/outputTest.pkl", 'wb'))

# In case we need this

In [55]:
pickle.dump(truthDf, open("binaries/truthDf.pkl", 'wb'))
pickle.dump(instancesDf, open("binaries/instancesDf.pkl", 'wb'))
